In [20]:
import requests
import base64
import json

In [21]:
def get_pinned_repos(username, token):
    url = "https://api.github.com/graphql"
    headers = {"Authorization": f"Bearer {token}"}
    query = """
    query {
      user(login: "%s") {
        pinnedItems(first: 6, types: [REPOSITORY]) {
          edges {
            node {
              ... on Repository {
                name
              }
            }
          }
        }
      }
    }
    """ % username
    response = requests.post(url, headers=headers, json={'query': query})

    if response.status_code == 200:
        data = response.json()
        repo_names = [edge['node']['name'] for edge in data['data']['user']['pinnedItems']['edges']]
        return repo_names
    else:
        return f"Failed to get repos, status code: {response.status_code}"

def get_repo_languages(username, repo):
    url = f"https://api.github.com/repos/{username}/{repo}/languages"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to get languages, status code: {response.status_code}"

def get_languages_for_repos(username, repos):
    languages_for_repos = {}
    for repo in repos:
        languages = get_repo_languages(username, repo)
        languages_for_repos[repo] = languages
    return languages_for_repos

# Fetch pinned repos
pinned_repo_names = get_pinned_repos("Sanchay-T", "")

# Get languages for each repo
languages_in_repos = get_languages_for_repos("Sanchay-T", pinned_repo_names)

# Print the languages for each repo
for repo, languages in languages_in_repos.items():
    print(f"Repo: {repo}, Languages: {languages}")
data_str = ""
for repo, languages in languages_in_repos.items():
    data_str += f"Repository {repo} uses the following languages: {', '.join(languages.keys())}.\n"


Repo: E-commerce, Languages: {'CSS': 871920, 'JavaScript': 787374, 'PHP': 197376, 'HTML': 37398, 'Hack': 14086}
Repo: Sales-Inventory-Management, Languages: {'Java': 51396}
Repo: Cancer_Prediction, Languages: {'HTML': 3405147, 'Jupyter Notebook': 2786752}
Repo: Facial-Recognition, Languages: {'Python': 3809}
Repo: StockPrice_Prediction, Languages: {'Jupyter Notebook': 749640}


In [22]:
import openai

openai.api_key = ''
prompt = f"""
{data_str}

For each repository, identify the primary programming language used. Determine which languages are most common across all repositories. Based on the most frequently used language, infer the primary programming skills of the user.
"""

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  temperature=0.3,
  max_tokens=300
)

print(response.choices[0].text.strip())


The primary programming language used across all repositories is Python. This indicates that the user has strong programming skills in Python, as well as familiarity with HTML, Jupyter Notebook, JavaScript, PHP, Hack, and Java.
